# Training PPO

At least 40GB VRAM is needed for this training

## PPO

In [1]:
from trl import PPOConfig

config = PPOConfig(
    model_name="tinyllama",
    learning_rate=1e-6,
    log_with= 'wandb',
    batch_size = 128,
    mini_batch_size = 16,
    gradient_accumulation_steps = 8,
    
)

/opt/conda/envs/llmrl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer
# from unsloth import FastLanguageModel

from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer

# model = AutoModelForCausalLMWithValueHead.from_pretrained("checkpoints/tinyLlama-GSM8K-10epochs")
# tokenizer = AutoTokenizer.from_pretrained("checkpoints/tinyLlama-GSM8K-10epochs")

model = AutoModelForCausalLMWithValueHead.from_pretrained("unsloth/tinyllama")
#force to pad right since we will need to train the tokenizer. pad left is for inference only
tokenizer = AutoTokenizer.from_pretrained("unsloth/tinyllama",padding_side="right", use_fast =False)

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "checkpoints/tinyLlama-GSM8K-10epochs", # "unsloth/tinyllama" for 16bit loading
#     max_seq_length = 2048,
#     dtype = None,
#     load_in_4bit = False,
# )

tokenizer.pad_token = tokenizer.eos_token

In [3]:
from transformers import pipeline
# from unsloth import FastLanguageModel
# # reward_model = pipeline("text-classification", model="lvwerra/distilbert-imdb")
# critic, critic_tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "/home/jianingqi/LLMRL/checkpoints/llama3-8b-critic-lora-4-29", # "unsloth/tinyllama" for 16bit loading
#     max_seq_length = 2048,
#     dtype = None,
#     load_in_4bit = False,
#     # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
# )
# FastLanguageModel.for_inference(critic) # Enable native 2x faster inference
# critic_tokenizer.padding_side = "left" # Padding side for faster inference

from transformers import AutoModelForCausalLM, AutoTokenizer
critic = AutoModelForCausalLM.from_pretrained("checkpoints/llama3-critic-full")
critic_tokenizer = AutoTokenizer.from_pretrained("checkpoints/llama3-critic-full",  padding_side='left', use_fast =False)

critic.eval()
critic.to("cuda")

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.28it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [4]:
# critic.save_pretrained_merged("checkpoints/llama3-critic-full", critic_tokenizer, save_method = "merged_16bit",)

In [5]:
prompt = """
### Input:
{}

### Response:
"""
from datasets import load_dataset

dataset = load_dataset("gsm8k", 'main', split='train')


# EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(example):
    questions = example['question']
    answers = example['answer']
    output_texts = []
    for questions, answers in zip(questions, answers):
        output_text = prompt.format(questions)
        output_texts.append(output_text)
    return { "query" : output_texts, }
pass

dataset = dataset.map(formatting_prompts_func,  batched=True)
dataset = dataset.remove_columns(["question", "answer"])

def tokenize(sample):
    sample["input_ids"] = tokenizer.encode(sample["query"], padding="max_length", truncation=True, max_length=512)
    return sample

dataset = dataset.map(tokenize, batched=False)

In [6]:
from trl import PPOTrainer

ppo_trainer = PPOTrainer(
    model=model,
    config=config,
    dataset=dataset,
    tokenizer=tokenizer,
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jq394 (neurorunner). Use `wandb login --relogin` to force relogin


In [7]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 40,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 256,
    "use_cache": True,
    "temperature": 0.5,
}

In [8]:
def answer_process(q, r):
    question = q.split('### Response:')[0].split('\n### Input:\n')[1]
    responses = r.split('\n')
    for response in responses:
        question += response + " ки \n"
    return question

def count_lines(r):
    responses = r.split('\n')
    return len(responses)
    

In [9]:
import wandb
import os

wandb.login()
os.environ['WANDB_PROJECT'] = 'LLMRL'

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [11]:
from tqdm import tqdm
import torch
# from transformers.utils import logging

# logging.set_verbosity_info()

good_token = ' +'
bad_token = '-'
step_tag = ' ки'

candidate_tokens = critic_tokenizer.encode(f"{good_token} {bad_token}")[1:] # [648, 387]
step_tag_id = critic_tokenizer.encode(f"{step_tag}")[-1] # 12902
print(candidate_tokens)
print(step_tag_id)
    
epochs = 10
for epoch in tqdm(range(epochs), "epoch: "):
    for batch in tqdm(ppo_trainer.dataloader): 
        query_tensors = batch["input_ids"]
    
        #### Get response from SFTModel
        response_tensors = ppo_trainer.generate(query_tensors, **generation_kwargs)
        batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]
    
        #### Compute reward score
        texts = [answer_process(q,r) for q, r in zip(batch["query"], batch["response"])]
        batch_count = [count_lines(r) for q, r in zip(batch["query"], batch["response"])]
        
        #reward model in and output
        inputs = critic_tokenizer(texts, padding="max_length", truncation=True, max_length=512, return_tensors="pt").to("cuda")
        logits = critic(**inputs).logits[:,:,candidate_tokens]
        scores = logits.softmax(dim=-1)[:,:,0] 
        step_scores = scores[inputs['input_ids'] == step_tag_id]
        
        probability_index = 0
        rewards = []
        for i, count in enumerate(batch_count):
            answer_probs = step_scores[probability_index:probability_index+count]
            if answer_probs:
                # answer_prob = min(answer_probs)
                answer_prob = torch.tensor(answer_probs).prod().item()
                rewards.append(answer_prob)
            else:
                print('There is a length mismatch between input and score. Probably the critic tokenizer caused the problem')
            
        
        # pipe_outputs = reward_model(texts)
        # rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]
    
        #### Run PPO step
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
        ppo_trainer.log_stats(stats, batch, rewards)


[489, 482]
116624


epoch:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:

#### Save model
ppo_trainer.save_pretrained("tinyllama-ppo-10")